### Import libraries and data

In [1]:
import pandas as pd
import csv
import re
import numpy as np

In [2]:
vols = pd.read_csv("volumes.csv")
vols = vols.drop("Unnamed: 0", axis=1)

In [3]:
temps = pd.read_csv("temperatures.csv")
temps = temps.drop("Unnamed: 0", axis=1)

In [4]:
unitformula = pd.read_csv("scaledformulae.csv")
unitformula = unitformula.drop("Unnamed: 0", axis=1)

### Setting models

In [5]:
from sklearn import linear_model, svm, kernel_ridge
from scipy.optimize import nnls 
from sklearn.metrics import mean_squared_error
regVols = linear_model.LinearRegression()
regAlpha = linear_model.LinearRegression()
#regVols = linear_model.ElasticNet(positive = True)

### RMSD function

In [6]:
def getRMSD (y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

#w1 = [(3),(5),(10),(1)]
#w2 = [(2),(4),(6),(8)]
#getRMSD(w1,w2)

### Percentage function

In [7]:
def percCalc (row):
    predictedvolume = row["sum"]
    vols = row["Volume"]
    percentage = predictedvolume / vols
    if (percentage >=0.95) and (percentage <=1.05):
        return 1
    else:
        return 0

### Loop algorithm

In [ ]:
alpha = 0.00
elements = unitformula.columns.values
unitformula["Keep"] = 1
weights = []
alphas =[]
for chunk in zip(unitformula.groupby(np.arange(len(unitformula)) // 10000),vols.groupby(np.arange(len(vols)) // 10000),temps.groupby(np.arange(len(temps)) // 10000)):
    print "Chunk: ", chunk[0][0]
    prevalpha = 1
    RMSD = 10
    chunkformula = chunk[0][1]
    chunkvol = chunk[1][1]
    chunktemp = chunk[2][1]
    keepidx = chunkformula.index.tolist()
    i = 0
    while abs(prevalpha-alpha) > 0.00001 or (RMSD > 1) :
        print "Iteration:", i
        regVols.fit(chunkformula[chunkformula.Keep == 1], chunkvol.loc[keepidx].div(chunktemp.loc[keepidx].Temperature*alpha+1, axis="index"))
        if i > 0:
            prevw = w#[0]
            #print "old w: ", prevw
        w = regVols.coef_[0]
        #w, rnorm = nnls(np.matrix(chunkformula[chunkformula.Keep == 1]),np.squeeze(np.array(chunkvol.loc[keepidx].div(chunktemp.loc[keepidx].Temperature*alpha+1, axis="index"))))
        if i > 0:
            RMSD = getRMSD(prevw,w)#[0]
            print "New w", zip(elements, w)
            print "RMSD: ", RMSD
        x2=pd.DataFrame()
        for line in zip(elements, w): #[0]
            x2[line[0]] = chunk[0][1][line[0]].apply(lambda x: x*line[1])
        x2["sum"] = x2.sum(axis=1)
        #if i>0:
            #print x2["sum"]
        vol2 = chunkvol.loc[keepidx].div(x2["sum"].loc[keepidx], axis="index") - 1
        regAlpha.fit(chunktemp.loc[keepidx], vol2)
        if i > 0:
            prevalpha = alpha
            print "Previous Alpha: ", prevalpha
        alpha = regAlpha.coef_[0][0]
        #alpha, rnorm2 = nnls(np.matrix(chunktemp.loc[keepidx]),np.squeeze(np.array(vol2)))
        print "Alpha: ", alpha
        print "deltaAlpha: ", abs(prevalpha-alpha)
        #print "dAlpha: ", abs(prevalpha - alpha)
        chunkformula["Keep"] = pd.concat([x2,vols], axis=1).apply(percCalc, axis=1)
        keepidx = chunkformula.Keep[chunkformula.Keep == 1].index.tolist()
        #print "Element average volume (cubic angstroms)", zip(elements,w[0])
        i +=1
        if (len(keepidx) <= 1) or (i > 200):
            print "Failure"
            break
    weights.append(w)#[0])
    alphas.append(alpha)

In [ ]:
#[H: 5.08, Li: 22.6, Be: 36, B: 13.24, C: 13.87, N: 11.8,\
#O: 11.39, F: 11.17, Na: 26, Mg: 36, Al: 39.6, Si: 37.3, P: 29.5,\
#S: 25.2, Cl: 25.8, K: 36, Ca: 45, Sc: 42, Ti: 27.3, V: 24, Cr: 28.1,\
#Mn: 31.9, Fe: 30.4, Co: 29.4, Ni: 26, Cu: 26.9, Zn: 39, Ga: 37.8, Ge: 41.6, As: 36.4,\
#Se: 30.3, Br: 32.7, Rb: 42, Sr: 47, Y: 44, Zr: 27,\
#Nb: 37, Mo: 38, Tc: 38, Ru: 37.3, Rh: 31.2, Pd: 35, Ag: 35, Cd: 51, In: 55, Sn: 52.8,\
#Sb: 48, Te: 46.7, I: 46.2, Xe: 45, Cs: 46, Ba: 66, La: 58, Ce: 54, Pr: 57, Nd: 50, Sm: 50,\ 
#Eu: 53, Gd: 56, Tb: 45, Dy: 50, Ho: 42, Er: 54, Tm: 49, Yb: 59]

headers = ["Ru", "Rb", "Rh", "Be", "Ba", "Bk", "Br", "H", "P", "Ge", "Gd", "Ga", "Pr", "Pu", "C",\
"Pd", "Cd", "Ho", "Mg", "Mo", "Mn", "O", "S", "Eu", "Zr", "Er", "Ni",\
"Na", "Nb", "Nd", "Fe", "B", "F", "Sr", "N", "Si", "Sn", "Sm", "V", "Sc", "Sb", "Se", "Co",\
"Cl", "Ca", "Ce", "Xe", "Cs", "Cr", "Cu", "La", "Li", "Tm", "Ti", "Te", "Tb", "Tc", "Yb", "Dy",\
"I", "Y", "Ag", "Al", "As", "In"]

for elements in w:
    if elements not in headers:
        print np.where(w)

#hofmw = [37.3, 42, 31.2, 36, 13.24, 66, 32.7, 5.08, 29.5, 41.6, 56, 37.8, 57, 13.87, 35, 51, 42, 36, 36, 38, 31.9, 11.39, 25.2, 39, 53, 27, 54, 26
#26, 37, 50, 30.4, 13.24, 11.17, 47, 11.8, 37.3, 52.8, 50, 24, 42, 48, 30.3, 29.4, 25.8, 45, 54, 45, 46, 28.1, 26.9, 58, 22.6, 49
#27.3, 46.7, 45, 38, 59, 50, 46.2, 44, 35, 39.6, 36.4, 55]

Early attempts at gathering the elements used by Hofmann. 
The reason for this was to see if the results would improve using just the elements Hofmann did & also to be able to directly compare

The "headers" list was ordered as the same as the one used by me in the unitformulas csv file.

The inital dictionary is the Hofmann elements & corresponding volumes; the last list "hofmw" is just the volumes.